In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from lightfm import LightFM
from lightfm.data import Dataset

# --- 1. 데이터 준비 (학습용) ---
# 3명의 기존 사용자(0, 1, 2)와 4개의 아이템(0, 1, 2, 3)
# (사용자 ID, 아이템 ID)
interactions_data = [
    (0, 0), (0, 2),  # 사용자 0
    (1, 1), (1, 3),  # 사용자 1
    (2, 0), (2, 1)   # 사용자 2
]

# 사용자 특징: (사용자 ID, [특징 리스트])
# 사용자 0: '20s', 'F'
# 사용자 1: '20s', 'M'
# 사용자 2: '30s', 'M'
user_features_data = [
    (0, ['20s', 'F']),
    (1, ['20s', 'M']),
    (2, ['30s', 'M'])
]

# 아이템 특징: (아이템 ID, [특징 리스트])
# 아이템 0: 'Action'
# 아이템 1: 'Comedy'
# 아이템 2: 'Sci-Fi'
# 아이템 3: 'Comedy'
item_features_data = [
    (0, ['Action']),
    (1, ['Comedy']),
    (2, ['Sci-Fi']),
    (3, ['Comedy'])
]

# LightFM의 Dataset 객체를 사용해 특징과 ID를 매핑합니다.
dataset = Dataset()

# 사용자 ID, 아이템 ID, 그리고 모든 특징(feature)들을 등록
dataset.fit(
    users=(x[0] for x in user_features_data),
    items=(x[0] for x in item_features_data),
    user_features=(f for x in user_features_data for f in x[1]),
    item_features=(f for x in item_features_data for f in x[1])
)

# 상호작용 행렬 (Interactions Matrix) 생성
(interactions, weights) = dataset.build_interactions(interactions_data)

# 특징 행렬 생성
# (여기서 ID 자체도 특징으로 포함시킬 수 있지만, 
# 콜드스타트를 명확히 보기 위해 ID는 제외하고 순수 특징만 사용합니다)
user_features = dataset.build_user_features(user_features_data)
item_features = dataset.build_item_features(item_features_data)


# --- 2. 모델 학습 ---
# 모델 초기화 (WARP: 암묵적 피드백(implicit)에 적합한 손실 함수)
model = LightFM(loss='warp', no_components=10, random_state=42)

# 학습! (사용자 특징과 아이템 특징을 함께 넣어줍니다)
model.fit(
    interactions,
    user_features=user_features,
    item_features=item_features,
    epochs=20,
    num_threads=4,
    verbose=False
)

print("--- 모델 학습 완료 ---")


# --- 3. 🚨 콜드 스타트 사용자 추론 (핵심!) ---

# '새로운 사용자 (New User)'를 가정합니다.
# 이 사용자는 학습(fit)에 사용된 적이 없습니다.
# 특징: ['30s', 'M'] (이 특징들은 학습 데이터에 존재했음)
new_user_features_list = ['30s', 'M']

# 1. 이 새로운 사용자를 위한 '특징 행렬'을 직접 만듭니다.
#    (1명의 사용자, 전체 사용자 특징 개수) 크기의 행렬

# dataset.mapping()을 통해 특징이 내부적으로 몇 번 인덱스에 매핑되었는지 가져옵니다.
# mapping[2]가 사용자 특징 맵(dictionary)입니다.
user_feature_map = dataset.mapping()[2]

# '30s'와 'M'의 인덱스를 찾습니다.
new_user_feature_indices = [user_feature_map[f] for f in new_user_features_list]

# 전체 사용자 특징의 개수
n_user_features = user_features.shape[1]

# 1xN 크기의 CSR 행렬 생성 (새로운 사용자 1명)
# (데이터, (행 인덱스, 열 인덱스)), shape
data = [1] * len(new_user_feature_indices) # [1, 1]
rows = [0] * len(new_user_feature_indices) # [0, 0] (0번째 행=새 사용자)
cols = new_user_feature_indices           # [index_of_'30s', index_of_'M']

new_user_features_matrix = csr_matrix(
    (data, (rows, cols)),
    shape=(1, n_user_features)
)

print(f"\n새로운 사용자 특징: {new_user_features_list}")
# print("새로운 사용자 특징 행렬 (희소):\n", new_user_features_matrix)


# 2. 예측 (Predict)
# 모든 아이템 (0, 1, 2, 3)에 대한 점수를 계산
all_item_ids = np.arange(interactions.shape[1])

# ⭐️ 여기가 핵심입니다 ⭐️
# user_ids=[0] : 우리가 방금 만든 'new_user_features_matrix'의 0번째 행을 사용하라는 의미
# user_features=... : '기존' 학습된 특징 행렬이 아닌, 
#                     '새로 만든' 콜드 스타트용 특징 행렬을 전달
scores = model.predict(
    # user_ids = [0],
    user_ids=np.repeat(0, len(all_item_ids)), 
    item_ids=all_item_ids,
    user_features=new_user_features_matrix,
    item_features=item_features
)

# 3. 결과 확인
item_id_map = dataset.mapping()[1]
# 아이템 ID -> 아이템 이름(특징)으로 보기 쉽게 변환
item_names = [item_features_data[i][1][0] for i in all_item_ids] 

# 점수가 높은 순으로 정렬
sorted_indices = np.argsort(-scores)

print("\n--- '새로운 사용자' (30s, M) 추천 결과 ---")
for idx in sorted_indices:
    print(f"  아이템 {idx} ({item_names[idx]}): {scores[idx]:.4f}")


# --- 4. 비교: 기존 사용자 (30s, M) ---
# 기존 사용자 '2' (Charlie)도 ['30s', 'M'] 특징을 가졌습니다.
# 이 사용자의 추천 결과와 비교해봅시다.
# (이때는 '기존' user_features 행렬을 사용합니다)
existing_user_scores = model.predict(
    user_ids=2, # 사용자 2번
    item_ids=all_item_ids,
    user_features=user_features, # 기존 행렬
    item_features=item_features
)

print("\n--- '기존 사용자 2' (30s, M) 추천 결과 (비교용) ---")
sorted_indices_existing = np.argsort(-existing_user_scores)
for idx in sorted_indices_existing:
    print(f"  아이템 {idx} ({item_names[idx]}): {existing_user_scores[idx]:.4f}")

c:\Users\USER\Documents\github_personal\study\07_AI\lightfm\.venv\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


: 